In [ ]:
# Author: M. Riley Owens (GitHub: mrileyowens)

# This notebook calculates the depth of the rest-LyC HST/WFC3 F275W image
# of the Sunburst Arc

In [22]:
import os

from datetime import datetime, timezone

import numpy as np

from photutils.utils import ImageDepth
from photutils.segmentation import SourceFinder

from astropy.io import fits
from astropy.wcs import WCS

In [25]:
def measure():

    '''
    This function calculates the depth of the F275W observations of the Sunburst Arc
    '''

    # Establish common directories
    home = os.getcwd()
    data = f'{home}/data'
    results = f'{home}/results'
    
    # Set the file path to the F275W observation
    f275w_file = f'{data}/hst/V5.0_PSZ1G311.65-18.48_F275W_0.03g0.6_crsc1.2_0.7crsn3.5_3.0_drc_sci.fits'

    # Get the FITS header of the F275W observation
    f275w_header = fits.getheader(f275w_file)

    # Retrieve the WCS of the F275W observation
    f275w_wcs = WCS(f275w_header)

    # Get the inverse sensitivity of the image from the FITS header of the file
    inverse_sensitivity = f275w_header['PHOTFLAM']

    # The pivot wavelength of the F275W filter in angstroms
    w_pivot = 2709.7

    # Get the data array of the F275W observation for a core area of the drizzled image, where all the
    # frames commonly overlap. We want to calculate the depth of the image from this portion of the image
    # because the frames also commonly overlap on the Sunburst Arc, so it is the best representation of 
    # the depth achieved on the Sunburst Arc
    f275w = fits.getdata(f275w_file)[3500:5000, 3500:5500]

    # Convert the data from counts/s to erg/s/cm^2/Å
    f275w = f275w * inverse_sensitivity

    # Convert the data to a frequency-space flux density (erg/s/cm^2/Hz) based on the pivot wavelength of
    # the WFC3/UVIS F275W filter
    f275w = f275w * w_pivot**2 / (3e8 * 1e10)

    # Create a mask representing the location of any NaNs in the image
    nan_mask = np.isnan(f275w)

    # Create a mask of the sources in the image
    finder = SourceFinder(npixels=9, progress_bar=False)
    mask = finder(np.where(nan_mask, 0, f275w), abs(np.nanmedian(f275w))).make_source_mask()

    # Set the number of iterations to measure the limiting flux density
    iters = 1000

    # Determine the depth of the image at a significance of 5σ, using AB magnitudes. Use apertures with radius corresponding
    # to the approximate FWHM of the image PSF. Since the apertures use the summed flux densities within the aperture rather
    # than the average or typical flux density, there is a strong dependence between the estimated depth of the image and the
    # size of the apertures. I think it is most appropriate to set the aperture size to the effective size of an idealized 
    # point source in the image to represent the faintest point source the image could detect
    depth = ImageDepth(1.25, nsigma=5., mask_pad=2., napers=1000, niters=iters, seed=0, zeropoint=-48.6, progress_bar=False)
    limits = depth(f275w, (nan_mask | mask))

    # Get a list of arrays of the aperture objects used to make the measurements each iteration
    apertures = depth.apertures

    # For each set of apertures used every iteration
    for i, aperture_set in enumerate(apertures):

        # Convert the apartures from pixel coordinates to celestial coordinates and convert the array to a string representation
        apertures[i] = aperture_set.to_sky(f275w_wcs).positions.to_string(style='decimal')

    # As mentioned below, for unclear reasons, not all the iterations used 1000 apertures, so pad the missing entries with NaNs
    apertures = np.array([np.pad(arr, (0, iters - len(arr)), constant_values='nan nan') for arr in apertures])

    # Pad the summed flux densities measured from each aperture for each iteration with NaNs, since not every iteration has 1000 
    # apertures. It is unclear to me (M. Riley Owens) why photutils does not appear to create 1000 apertures when napers=1000. 
    # Testing with different numbers of apertures showed that this problem is not specific to the choice of number of apertures 
    # per iteration.
    flux_densities = depth.fluxes
    flux_densities = np.array([np.pad(arr, (0, iters - len(arr)), constant_values=np.nan) for arr in flux_densities])

    # Set the header of the flux density measurements output file
    header = 'Flux density measurements used to assess the depth of the HST/WFC3 F275W image of the Sunburst Arc.\n' \
        + f'Created by limit.ipynb on {datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")}\n' \
        + '\n' \
        + 'Each row represents the set of flux density measurements at each iteration. At each iteration, the code randomly generates \n' \
        + '1000 circular apertures of fixed radius from the unmasked portions of the F275W image (masking out NaNs and sources) and \n' \
        + f'sums the flux density in each aperture. The flux densities have units of erg/s/cm^2/Hz, assuming a pivot wavelength of {w_pivot} Å.\n' \
        + '\n' \
        + f'Average 5σ flux density limit calculated from the iterations (erg/s/cm^2/Hz, AB magnitude): {limits[0], limits[1]}\n'

    # Save the measured flux densities
    np.savetxt(f'{results}/lyc_depth/f275w_depth_flux_density_measurements.txt', flux_densities, delimiter=' ', header=header, comments='# ', encoding='utf-8')

    # Set the header of the output apertures file
    header = 'Apertures used to assess the depth of the HST/WFC3 F275W image of the Sunburst Arc.\n' \
        + f'Created by limit.ipynb on {datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")}\n' \
        + '\n' \
        + 'Each row represents the set of circular apertures at each iteration. At each iteration, the code randomly generates \n' \
        + '1000 circular apertures of fixed radius (0.0375 arcseconds) from the unmasked portions of the F275W image (masking \n' \
        + 'out NaNs and sources). The listed coordinates appear like \'RA DEC\' and are the center of the apertures in the ICRS \n' \
        + 'celestical coordinate frame.\n'

    # Save the apertures
    np.savetxt(f'{results}/lyc_depth/f275w_depth_apertures.txt', apertures, fmt='%-s', delimiter=', ', header=header, comments='# ', encoding='utf-8')

In [26]:
measure()